# Complementaria Semana 2: Introducción a la simulación de Montecarlo

El modelado estocástico permite representar sistemas donde intervienen fenómenos aleatorios y la información disponible no es completamente determinista. En ingeniería y ciencias aplicadas esto incluye desde la variabilidad en mediciones y tiempos hasta la ocurrencia de eventos como fallas, demandas o llegadas. Para describir esa incertidumbre se utilizan distribuciones de probabilidad que asignan pesos a los posibles valores de una variable aleatoria. En contextos discretos, como el resultado de dados o el conteo de eventos, utilizamos funciones de probabilidad; mientras que en situaciones continuas, como tiempos de procesamiento, tiempos de espera en sistemas de servicio, consumo de recursos o demanda continua de productos, recurrimos a funciones de densidad. Entre las distribuciones más utilizadas se encuentran la Poisson para conteos, la Exponencial para tiempos entre eventos y la Normal, la cual tomará mayor relevancia con el Teorema Central del Límite.

Cuando el análisis es complejo, la simulación de Monte Carlo se convierte en una herramienta fundamental. El procedimiento consiste en generar datos de acuerdo con las distribuciones asumidas, computar indicadores de interés como medias, probabilidades o percentiles, y aproximar sus valores mediante repetición. La precisión mejora conforme aumenta el número de simulaciones, y el Teorema del Límite Central justifica que los promedios obtenidos mediante simulación se comporten aproximadamente como una variable normal, aun si la distribución original no lo es. Esto permite cuantificar la incertidumbre de las estimaciones mediante errores estándar e intervalos de confianza, fortaleciendo la capacidad de tomar decisiones informadas en presencia de aleatoriedad.

En esta complementaria nos enfocamos en modelar incertidumbre puramente aleatoria, asociada al azar inherente del fenómeno (lanzamiento de dados). Más adelante en el curso aparecerán otros tipos de incertidumbre, como la asociada a información incompleta o a decisiones humanas, que requerirán herramientas adicionales.

## Ejercicio guiado: Máximo de dos dados y simulación Monte Carlo


Analizaremos el comportamiento de una variable aleatoria definida a partir del lanzamiento de dos dados justos, y luego validaremos nuestros resultados teóricos mediante simulación. Para cerrar, utilizaremos el Teorema Central de Límite para observar cómo los promedios de variables aleatorias se aproximan a una distribución Normal.

### Contexto

Suponga que lanzamos dos dados justos e independientes. Cada dado puede tomar valores del 1 al 6 con igual probabilidad. Definimos la variable aleatoria:


$$
M = \max(D_1, D_2)
$$

donde $D_1$ y $D_2$ son los resultados de cada dado. Nos interesa estudiar la distribución de $M$, calcular sus estadísticas y validar los resultados usando simulación Monte Carlo.


### Objetivos

- Derivar la función de probabilidad (pmf) de $M$ de forma teórica
- Simular y comparar teoría vs resultados Monte Carlo
- Calcular media y varianza teóricas y empíricas
- Observar el Teorema del Límite Central aplicándolo a promedios de $M$




### Parte A — Deriva la pmf teóricamente

Existen 36 posibles resultados $(D_1, D_2)$, todos igual de probables. Para $m = \{1,2,3,4,5,6\}$:

$$
P(M=m) = \left(\frac{m}{6}\right)^2 - \left(\frac{m-1}{6}\right)^2 = \frac{2m-1}{36}
$$

Esto genera la siguiente tabla:

| m | P(M = m) |
|---|---------|
| 1 | 1/36 |
| 2 | 3/36 |
| 3 | 5/36 |
| 4 | 7/36 |
| 5 | 9/36 |
| 6 | 11/36 |


### Parte B — Validación por simulación Monte Carlo 
A continuación simularemos muchos pares de lanzamientos, calcularemos el máximo en cada uno, y compararemos con la distribución teórica.

In [ ]:
# importar librerías
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Se fija una semilla para garantizar reproducibilidad de los resultados.
# Esto no elimina la aleatoriedad del fenómeno, sino que permite replicar
# exactamente el mismo experimento computacional.

rng = np.random.default_rng(0)    # 0) generador con semilla (reproducible)
N = 200000                        # 1) número de simulaciones

# 2) creación de las muestras de los dos dados y cálculo del máximo
d1 = rng.integers(1, 7, size=N)   # enteros en [1,6]
d2 = rng.integers(1, 7, size=N)
M  = np.maximum(d1, d2)

# 3) pmf empírica
#    3.1 cuenta ocurrencias forzando 7 posiciones (índices 0..6)
#    3.2 elimina el índice 0 (no hay resultado 0), quedan valores 1..6
counts  = np.bincount(M, minlength=7)[1:]
#    3.3 divide por N para obtener frecuencias (probabilidades)
pmf_emp = counts / N

# 4) pmf teórica (derivada en la parte A): P(M=m) = (2m - 1)/36 para m=1..6
m       = np.arange(1, 7)
pmf_the = (2*m - 1) / 36

# 5) impresión de resultados
#    5.1 pmf empírica
print("pmf empírica:", np.round(pmf_emp, 4))
#    5.2 pmf teórica
print("pmf teórica :", np.round(pmf_the, 4))
#    5.3 error absoluto componente a componente
print("error abs   :", np.round(np.abs(pmf_emp - pmf_the), 5))


pmf empírica: [0.0282 0.0817 0.1403 0.1946 0.2503 0.3049]
pmf teórica : [0.0278 0.0833 0.1389 0.1944 0.25   0.3056]
error abs   : [0.00042 0.00165 0.00137 0.00017 0.00033 0.00064]


Las pequeñas discrepancias observadas entre la distribución teórica y la empírica se deben al error muestral propio de trabajar con un número finito de simulaciones. A medida que el tamaño de la muestra aumenta, estas diferencias tienden a desaparecer, lo cual es consistente con la Ley de los Grandes Números, que garantiza la convergencia de las estimaciones empíricas hacia sus valores teóricos.

#### Profundización: resuelva estas preguntas ¿cómo cambia el código?

1) **¿Por qué usar una semilla en simulación?**  
   - Experimento: ejecute dos veces la simulación con la **misma semilla** y luego con **semillas distintas**.  
   - ¿Qué observa en las pmf empíricas?  
   - **Qué cambiar en el código:** modifique `seed` en la llamada a `default_rng(...)` (por ejemplo, `0` y `1`) y compare las salidas.

2) **¿Qué pasaría si N fuera muy pequeño, como 100?**  
   - Experimento: reduzca el **tamaño de muestra** a `N = 100`.  
   - ¿Cómo cambian las frecuencias relativas frente a la pmf teórica?  
   - **Qué cambiar en el código:** establezca `N = 100` en la celda de simulación y observe el vector `pmf_emp` y su error absoluto.

3) **¿Por qué la aproximación mejora al aumentar N?**  
   - Experimento: evalúe el **error** entre la pmf empírica y la teórica para varios valores de `N` (por ejemplo: 50, 100, 200, 500, 1 000, 5 000, 10 000, 50 000, 100 000).  
   **Hint:** Use un ciclo for y guarde los resultados en un dataframe
   - Grafique **error vs N** en escala logarítmica y describa la tendencia.  
   - **Qué cambiar en el código:** cree un arreglo con distintos `N`, repita la simulación para cada uno usando la **misma semilla** y calcule la suma de diferencias absolutas `np.abs(pmf_emp - pmf_the).sum()`.


### Parte C — Media y varianza

$$
E[M] = \sum_{m=1}^6 m \cdot \frac{2m-1}{36} 
$$

$$
Var(M) = \sum_{m=1}^6 m^2 \cdot \frac{2m-1}{36} - (E[M])^2 
$$


In [ ]:
# Cálculo de E[M] y Var(M) empíricos
# 1) E[M] empírico: suma m * pmf_emp(m)
E_emp = (m * pmf_emp).sum()

# 2) E[M^2] empírico y luego Var(M) = E[M^2] - (E[M])^2
Var_emp = ( (m**2 * pmf_emp).sum() - E_emp**2 )

# 3) impresión
print("E[M] (emp) =", E_emp, "  Var(M) (emp) =", Var_emp)  

E[M] (emp) = 4.47195   Var(M) (emp) = 1.9680331975000023


#### Profundización: pregunta corta

1) ¿Se obtiene el **mismo** \(E[M]\) calculando con `pmf_emp` que usando el **promedio muestral**? ¿Por qué?